In [2]:
import pandas as pd
import numpy as np
import os
import copy
import json
from tqdm.auto import tqdm

%load_ext autoreload
%autoreload 2

In [3]:
num_factors = 100

data_folder = '../data_Megogo/'
new_data_folder = '../generated_data/'
model_folder = new_data_folder +'models/factor_'+str(num_factors)+'/'
dic_folder = new_data_folder+'dic/'

In [4]:
import export_modules
from app.Recommender_implicit_new import Implicit
from app.metrics_new import Metrics

In [5]:
train = pd.read_csv(new_data_folder+'train_first_level.csv')
train[['user', 'item']] = train[['user', 'item']].astype(str)
train.head(2)

,user,item,rating
0,21603820,9583642,0.0839
1,35636970,24645936,0.5990


In [6]:
with open(dic_folder+'test_dic.json', 'r') as f:
    test_dic = json.load(f) 

In [7]:
users_in_test = list(test_dic.keys())

In [8]:
rec = Implicit()
rec.fit_trainset(train)

In [9]:
dic_param = {'factors': num_factors, 'regularization':1, 'iterations':10, 'use_cg':True}
rec.fit_model(dic_param)

100%|██████████| 10.0/10 [00:25<00:00,  2.59s/it]


In [13]:
rec.dump_model(model_folder+'model_als')

Model has succesfuly been dumped!


# Predict for demonstration

In [23]:
k = 10
rec.set_k(k)

predict_df = rec.recommend_df(users_in_test[3:], filter_already_liked_items=False, return_scores=True)

100%|██████████| 187180/187180 [00:25<00:00, 7235.91it/s]


In [24]:
predict_df.columns = ['user_id', 'video_id', 'score']

In [25]:
predict_df.head(30)

,user_id,video_id,score
0,235599,21757334,0.929609
1,235599,6597685,0.912704
2,235599,33501500,0.878318
3,235599,4889342,0.824930
4,235599,22328483,0.800370
5,235599,14540399,0.799846
6,235599,3636342,0.783520
7,235599,10979465,0.771781
8,235599,13536958,0.769564
9,235599,15117563,0.750291


# Upload meta for filter

In [14]:
with open(dic_folder+'meta_type_dic.json', 'r') as f:
    meta_type_dic = json.load(f) 

In [15]:
with open(dic_folder+'train_dic.json', 'r') as f:
    train_dic = json.load(f) 

In [16]:
k = 80
rec.set_k(k)

predict_ = rec.recommend(users_in_test, filter_already_liked_items=False)

100%|██████████| 187183/187183 [00:37<00:00, 4990.01it/s]


In [17]:
predict_to_filter = predict_

predict_filtered = {}

for user, items in tqdm(predict_to_filter.items()):
    train_items = []
    try:
        train_items = train_dic[user]
    except:
        pass
    filtered_items = []
    for item in items:
        if meta_type_dic[item] == 'FILM':
            if item in train_items:
                continue
        filtered_items.append(item)
    predict_filtered[user] = filtered_items

In [18]:
predict_10_filter = {}
for user, items in tqdm(predict_filtered.items()):
    predict_10_filter[user] = items[:10]

In [19]:
# check if we have at least 10 items in recommend

for items in predict_10_filter.values():
    if len(items) < 10:
        print('error')

In [20]:
mt = Metrics(test_dic, predict_10_filter)
mt.Evaluate(k=10, metrics_list=['Precision', 'Recall', 'mMAP', 'mNDCG'])

,Precision@10,Recall@10,mMAP@10,mNDCG@10
0,0.0425413,0.113818,0.0577854,0.0937778


In [18]:
mt = Metrics(test_dic, predict_10_filter)
mt.Evaluate(k=10, metrics_list=['Precision', 'Recall', 'mMAP', 'mNDCG'])

,Precision@10,Recall@10,mMAP@10,mNDCG@10
0,0.0418297,0.112241,0.0584143,0.0937909


# Add factors for new users in test

In [21]:
test = pd.read_csv(new_data_folder+'test.csv')
test.head(2)

,user,item,rating
0,1088,20344436,5.0
1,28577,1960532,5.0


In [22]:
rec.add_fit_trainset(test)

factors extended


In [23]:
del test

In [24]:
n_factors = num_factors
user_factors = rec.model.user_factors
item_factors = rec.model.item_factors

u_columns = ['uf'+str(i) for i in range(n_factors)]
i_columns = ['if'+str(i) for i in range(n_factors)]

In [25]:
user_features_df = pd.DataFrame(user_factors, columns= u_columns)
user_features_df['user_id'] = user_features_df.index
user_features_df['user_id'] = user_features_df['user_id'].map(rec.inv_mapping_dict['user'])
user_features_df.head(2)

,uf0,uf1,uf2,uf3,uf4,uf5,uf6,uf7,uf8,uf9,...,uf91,uf92,uf93,uf94,uf95,uf96,uf97,uf98,uf99,user_id
0,-0.071293,0.559415,-0.072539,0.129628,-0.280565,0.059878,-0.283093,-0.060836,0.450761,-0.330126,...,0.242459,-0.129850,-0.201479,0.126578,-0.234366,0.084919,-0.537230,0.062307,0.434249,21603820
1,0.078816,-0.080879,0.154772,-0.018486,1.056839,0.269800,0.520433,0.593316,0.011278,-0.335845,...,0.323231,0.206529,0.454821,0.121880,-0.179581,-0.278110,-0.029591,0.093737,0.507237,35636970


In [ ]:
item_features_df = pd.DataFrame(item_factors, columns= i_columns)
item_features_df['primary_video_id'] = item_features_df.index
item_features_df['primary_video_id'] = item_features_df['primary_video_id'].map(rec.inv_mapping_dict['item'])
item_features_df.head(2)

## Save factors

In [27]:
user_features_df.to_csv(model_folder+'user_features.csv', index=None)
item_features_df.to_csv(model_folder+'item_features.csv', index=None)

# Save predict

In [28]:
def create_df_from_predict(dic_predict):
    user_res = []
    item_res = []
    for user, items in dic_predict.items():
        row_user = [user]*len(items)
        user_res.extend(row_user)
        item_res.extend(items)

    return pd.DataFrame({'user_id': user_res, 'primary_video_id': item_res})

In [29]:
df_predict = create_df_from_predict(predict_10_filter)
df_predict.to_csv(model_folder+'predict_10.csv', index=None)
df_predict.head(2)

,user_id,primary_video_id
0,1088,2677761
1,1088,3696132


In [30]:
predict_20_filter = {}
for user, items in tqdm(predict_filtered.items()):
    predict_20_filter[user] = items[:20]

In [31]:
df_predict = create_df_from_predict(predict_20_filter)
df_predict.to_csv(model_folder+'predict_20.csv', index=None)
df_predict.head(2)

,user_id,primary_video_id
0,1088,2677761
1,1088,3696132
